### Import Statements

In [3]:
from google.cloud import bigquery
from pprint import pprint
import os

### Utility Big Query Function

In [2]:
def query_big_query(query_string):
    client = bigquery.Client()
    query_job = client.query(query_string)

    results = query_job.result()  # Waits for job to complete.
    return results

In [4]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/jnapolitano/Projects/pmc-submission/creds.json"

In [6]:
def marketing_query():

    query_string = """
        SELECT fullVisitorId
        ,COUNT(DISTINCT visitId) AS num_sessions
        ,SUM(totals.pageviews) AS num_pageviews
        ,SUM(totals.hits) AS num_hits
        ,SUM(totals.timeOnSite) AS total_timeonsite
        ,MAX(CASE WHEN trafficSource.medium = '(none)' THEN 1 ELSE 0 END) AS medium_direct
        ,MAX(CASE WHEN trafficSource.medium = 'organic' THEN 1 ELSE 0 END) AS medium_organic
        ,MAX(CASE WHEN trafficSource.medium = 'referral' THEN 1 ELSE 0 END) AS medium_referral
        ,MAX(CASE WHEN trafficSource.medium NOT IN ('(none)', 'organic', 'referral') THEN 1 ELSE 0 END) AS medium_other
        ,MAX(CASE WHEN device.deviceCategory = 'mobile' THEN 1 ELSE 0 END) AS device_mobile
        ,MAX(CASE WHEN device.deviceCategory = 'desktop' THEN 1 ELSE 0 END) AS device_desktop
        ,MAX(CASE WHEN device.deviceCategory = 'tablet' THEN 1 ELSE 0 END) AS device_tablet
        ,SUM(totals.transactions) AS transactions
        FROM `bigquery-public-data.google_analytics_sample.ga_sessions_2017*`
        GROUP BY fullVisitorId
         """
    result = query_big_query(query_string)
    return result.to_dataframe()

df_first_try = marketing_query()

In [7]:
df_first_try

,fullVisitorId,num_sessions,num_pageviews,num_hits,total_timeonsite,medium_direct,medium_organic,medium_referral,medium_other,device_mobile,device_desktop,device_tablet,transactions
0,6129445072071455935,2,5,5,59,0,1,0,0,0,1,0,<NA>
1,3272613461636100905,1,5,5,99,0,1,0,0,1,0,0,<NA>
2,1186975933341262821,1,10,10,377,0,1,0,0,0,1,0,<NA>
3,597777595223011106,2,16,18,378,1,0,0,0,0,1,0,<NA>
4,1478676114594382999,1,45,50,1379,0,1,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
365286,761482459581686627,1,1,1,<NA>,0,1,0,0,0,1,0,<NA>
365287,9652254795109918600,1,1,1,<NA>,0,1,0,0,0,1,0,<NA>
365288,7027195094486020675,1,1,1,<NA>,1,0,0,0,0,1,0,<NA>
365289,7684322088319077058,1,3,3,89,1,0,0,0,0,1,0,<NA>


In [11]:
file_name = "marketing_data.csv"
cwd = os.getcwd()

outpath = os.sep.join([cwd,file_name])

df_first_try.to_csv(outpath)